In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import mbuild as mb
import foyer
from foyer import Forcefield

import mbuild.formats.charmm_writer as mf_charmm

import mosdef_cp2k_writer


import parmed as pmd
from constrainmol import ConstrainedMolecule
import unyt as u

from mosdef_cp2kmcpy.mc import MC
import setter
warnings.filterwarnings("ignore", category=DeprecationWarning)
 

In [3]:

hydrogen_res_name = 'H2'
FF_file_hydrogen = 'h2_ff.xml'
hydrogen_FF = Forcefield(forcefield_files=FF_file_hydrogen)
hydrogen_mb = mb.load('[HH]', smiles=True)

hydrogen_mb.name = hydrogen_res_name

hydrogen_typed=hydrogen_FF.apply(hydrogen_mb)



constrain_mol = ConstrainedMolecule(hydrogen_typed)
constrain_mol.solve()
hydrogen_optimized_mb=mb.clone(hydrogen_mb)
hydrogen_optimized_mb.xyz=constrain_mol.xyz/10
print(hydrogen_optimized_mb.xyz)

/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/foyer/forcefield.py:448: UserWarning: No force field version number found in force field XML file.
  warnings.warn(
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/foyer/forcefield.py:460: UserWarning: No force field name found in force field XML file.
  warnings.warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:687: UserWarning: No unitcell detected for pybel.Molecule [H][H]	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


[[0.09977617 0.00628678 0.00246344]
 [0.17387617 0.00628678 0.00246344]]


In [4]:
hydrogen_optimized_mb.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
#random box to be used in mosdef_charmm_writer
hydrogen_box_bias  = mb.fill_box(compound=[hydrogen_optimized_mb ],
                             n_compounds=[1] ,
                            box=[1.0, 1.0, 1.0] )

In [6]:
FF_Dict = {hydrogen_optimized_mb.name:FF_file_hydrogen }

residues_List = [hydrogen_optimized_mb.name ]




hydrogen_box_bias.save("hydrogen_bias_coord.xyz",overwrite=True)

print('Running: GOMC FF file, and the psf and pdb files for the biasing potential file')
mf_charmm.charmm_psf_psb_FF(hydrogen_box_bias ,
                            'hydrogen_bias',
                            FF_filename ="hydrogen_charmm_bias" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            bead_to_atom_name_dict = {},
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )

! rm *.pdb *.xyz

Running: GOMC FF file, and the psf and pdb files for the biasing potential file
write_gomcdata: forcefield_selection = {'H2': 'h2_ff.xml'}, residues = ['H2']
******************************

GOMC FF writing each residues FF as a group for structure_0
forcefield type from compound = {'H2': 'h2_ff.xml'}
coulomb14scale from compound = {'H2': 0.0}
lj14scale from compound = {'H2': 0.0}
unique_types = ['H_H2']
No urey bradley terms detected, will use angle_style harmonic
******************************

writing the GOMC force field file 
NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
forcefield_dict = {0: 'H_H2'}
******************************

write_charmm_psf file is running
write_charmm_psf: forcefield_selection = {'H2': 'h2_ff.xml'}, residues = ['H2']
******************************

No urey bradley terms detected
******************************

write_charmm_pdb file is running
write_charmm_pdb: residues == ['H2']
fix_residue = None
fix_residue_in_box = None
bead_

/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


In [11]:
molecule_list=[hydrogen_optimized_mb, hydrogen_optimized_mb]
box=mb.box.Box(lengths=[1,1,1])
#Comment why pressure is needed
q=MC(molecule_list=molecule_list,n_box=2,n_molecules_each_box=[[49,2],[1,2]], box_list=[box,box],cutoff=200,functional='PBE',
     basis_set={'N':'DZVP-MOLOPT-GTH','H':'DZVP-MOLOPT-GTH'}, periodicity=['XYZ']*2,ensemble='GEMC_NVT',seed=1,project_name='NH3_NVT_GEMC',restart='FALSE',pressure=1*u.bar)

In [12]:
q.mc_initialization()

scf_tolerance not specified, set as 1e-6
basis_set_filename not defined, set as BASIS_MOLOPT
potential_filename not specified, set as GTH_POTENTIALS
n_steps not specified, set as 1000
n_ff_moves not specified, set as 8
nswapmoves not specified, set as 640, will be ignore if n_box<2
output trajectory format set as XYZ
input_filename not specified, set as ['GEMC_NVT_box1.inp', 'GEMC_NVT_box2.inp']
output_filename not specified, set as NH3_NVT_GEMC_mc_output.out
temperature not defined, set as 298 K
You can change default settings in setter.mc_files


In [15]:

q.topology_filename=['hydrogen_bias.psf', 'hydrogen_bias.psf']

# move_probabilities=[pmavbmc,pmcltrans,pmhmc,pmswap,pmtraion,pmtrans,pmvolume]
#volume moves = PMVOLUME
# swap moves = PMSWAP - PMVOLUME
# AVBMC moves = PMAVBMC - PMSWAP - PMVOLUME
# an “inner” move = 1.0 - (PMAVBMC + PMSWAP + PMVOLUME)
#conformational changes = “inner” move percentage × PMTRAION
# molecular translation = “inner” move percentage × (PMTRANS - PMTRAION)
# molecular rotation = “inner” move percentage × (1.0 - PMTRANS - PMTRAION)

q.move_probabilities=[0,0.0,0,0.4,0.5,0.75,0.05]



# mol_probabilities=[[PMAVBMC_MOL,PMSWAP_MOL , PMTRAION_MOL, PMTRANS_MOL,PMROT_MOL],[]]
q.mol_probabilities=[[[1],[1],[1],[1],[1]],[[1],[1],[1],[1],[1]]]



#avbmc probabilities=[[AVBMC_ATOM,AVBMC_RMIN,AVBMC_RMAX,PBIAS],[]]
q.avbmc_probabilities=[[[1],[1],[1],[1]],[[1],[1],[1],[1]]]

q.charmm_potential_file='hydrogen_charmm_bias.inp'

In [16]:
setter.mc_files(q)

MC initial structure saved as ['NH3_NVT_GEMC_box1_initial.xyz', 'NH3_NVT_GEMC_box2_initial.xyz']
MC input file saved as GEMC_NVT_box1.inp
MC input file saved as GEMC_NVT_box2.inp
